In [1]:
import pandas as pd
data = pd.read_csv('titles.csv', error_bad_lines=False);
data_text = data[['headline_text']]
data_text['index'] = data_text.index
documents = data_text

/var/folders/nt/dpyj92ks2571pfbrjdswh68c0000gn/T/ipykernel_51094/1550564268.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv('titles.csv', error_bad_lines=False);


In [2]:
print(len(documents))
print(documents[:5])

34367
                                       headline_text  index
0  All Reliance Jio voice calls to any network in...      0
1  Fact Check: This is not a Jio tower set on fir...      1
2  Fact Check: No, Kerala has not banned Jio Inte...      2
3  China's Zhong Shanshan replaces Mukesh Ambani ...      3
4  Wildfire Breaks Out At Nagaland's Dzukou Valle...      4


In [3]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import *
from nltk.stem.porter import *
import numpy as np
np.random.seed(2022)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/hruthvik/nltk_data...


True

In [4]:
stemmer = PorterStemmer()
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3 and token!='says'and token!='saying' and token!='analytics' and token!='business' and token!='nestle' and token!='indian' and token!='mint' and token!='economic' and token!='times' and token!='india' and token!='news' and token!='titan' and token!='maruti' and token!='moneycontrol' and token!='today' and token!='mahindra' and token!='cnbctv' and token!='tata' and token!='titans' and token!='analyst' and token!='wipro' and token!='time' and token!='insight' and token!='nestlé' and token!='suzuki' and token!='reddit' and token!='hdfc'and token!='say' and token!='bloomberg' and token!='telangana' and token!='airtel' and token!='airtel' and token!='inside':
            result.append(lemmatize_stemming(token))
    return result

In [5]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/hruthvik/nltk_data...


True

In [6]:
doc_sample = documents[documents['index'] == 4310].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['247', 'CPSEs', 'earned', 'profit', 'of', 'Rs', '1.78', 'lakh', 'crore', 'in', 'FY19:', 'CAG', '-', 'Economic', 'Times']


 tokenized and lemmatized document: 
['cpse', 'earn', 'profit', 'lakh', 'crore']


In [7]:
processed_docs = documents['headline_text'].map(preprocess)
processed_docs[:10]

0                 [relianc, voic, call, network, free]
1          [fact, check, tower, amid, farmer, protest]
2    [fact, check, kerala, ban, internet, servic, s...
3    [china, zhong, shanshan, replac, mukesh, amban...
4    [wildfir, break, nagaland, dzukou, valley, muk...
5            [case, sebi, slap, fine, relianc, ambani]
6    [relianc, industri, chairman, fin, share, trad...
7    [relianc, infra, complet, sale, delhi, agra, t...
8           [indian, recharg, phone, onlin, ettelecom]
9    [hundr, bird, drop, dead, rome, year, firework...
Name: headline_text, dtype: object

In [8]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 call
1 free
2 network
3 relianc
4 voic
5 amid
6 check
7 fact
8 farmer
9 protest
10 tower


In [9]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [10]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

[(84, 1), (311, 1), (462, 1), (630, 1)]

In [11]:
bow_doc_4310 = bow_corpus[4310]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                               dictionary[bow_doc_4310[i][0]], 
bow_doc_4310[i][1]))

Word 84 ("crore") appears 1 time.
Word 311 ("lakh") appears 1 time.
Word 462 ("earn") appears 1 time.
Word 630 ("profit") appears 1 time.


In [12]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.4865976358029827),
 (1, 0.41303197443440254),
 (2, 0.42620501011248585),
 (3, 0.31056809075870895),
 (4, 0.5608244638773106)]


In [13]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=4, id2word=dictionary, passes=2, workers=2)

In [14]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.028*"power" + 0.016*"grid" + 0.011*"energi" + 0.010*"solar" + 0.009*"reddi" + 0.008*"plant" + 0.008*"industri" + 0.007*"steel" + 0.007*"texa" + 0.007*"tech"
Topic: 1 
Words: 0.019*"carandbik" + 0.015*"launch" + 0.013*"electr" + 0.008*"plan" + 0.008*"auto" + 0.007*"price" + 0.007*"vehicl" + 0.007*"smart" + 0.007*"year" + 0.006*"data"
Topic: 2 
Words: 0.037*"bank" + 0.016*"covid" + 0.011*"crore" + 0.011*"vaccin" + 0.010*"express" + 0.009*"standard" + 0.009*"financi" + 0.009*"stock" + 0.007*"year" + 0.007*"sensex"
Topic: 3 
Words: 0.017*"market" + 0.014*"energi" + 0.011*"standard" + 0.011*"covid" + 0.010*"stock" + 0.009*"power" + 0.008*"auto" + 0.008*"price" + 0.007*"invest" + 0.007*"storag"


In [15]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=4, id2word=dictionary, passes=3, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.008*"energi" + 0.007*"power" + 0.007*"profit" + 0.005*"sensex" + 0.005*"reddi" + 0.005*"nifti" + 0.005*"result" + 0.004*"market" + 0.004*"rise" + 0.004*"data"
Topic: 1 Word: 0.008*"tech" + 0.007*"stock" + 0.007*"bank" + 0.006*"launch" + 0.005*"compani" + 0.005*"standard" + 0.005*"rais" + 0.005*"covid" + 0.004*"develop" + 0.004*"vaccin"
Topic: 2 Word: 0.011*"intellig" + 0.010*"artifici" + 0.007*"power" + 0.007*"bank" + 0.006*"invest" + 0.006*"standard" + 0.006*"grid" + 0.005*"express" + 0.005*"market" + 0.005*"texa"
Topic: 3 Word: 0.007*"bank" + 0.006*"covid" + 0.005*"know" + 0.005*"employe" + 0.005*"launch" + 0.005*"express" + 0.005*"need" + 0.004*"year" + 0.004*"tech" + 0.004*"best"


In [16]:
corpus_tfidf

In [17]:
bow_corpus

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)],
 [(5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1)],
 [(6, 1), (7, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1)],
 [(16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1)],
 [(16, 1), (17, 1), (19, 1), (22, 1), (23, 1), (24, 1), (25, 1)],
 [(3, 1), (16, 1), (26, 1), (27, 1), (28, 1)],
 [(3, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1)],
 [(3, 1),
  (35, 1),
  (36, 1),
  (37, 1),
  (38, 1),
  (39, 1),
  (40, 1),
  (41, 1),
  (42, 1)],
 [(43, 1), (44, 1), (45, 1), (46, 1), (47, 1)],
 [(48, 1), (49, 1), (50, 1), (51, 1), (52, 1)],
 [(53, 1), (54, 1), (55, 1)],
 [(3, 1),
  (16, 1),
  (19, 1),
  (28, 1),
  (31, 1),
  (34, 1),
  (56, 1),
  (57, 1),
  (58, 1),
  (59, 1)],
 [(3, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1)],
 [(10, 1), (26, 1), (66, 1), (67, 1), (68, 1)],
 [(3, 1),
  (69, 1),
  (70, 1),
  (71, 1),
  (72, 1),
  (73, 1),
  (74, 1),
  (75, 1),
  (76, 1)],
 [(3, 1), (61, 1), (77, 1), (78, 1), (79, 1)],
 [(

In [30]:
import sklearn

# from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib notebook
from sklearn.linear_model import LogisticRegression
import sklearn.model_selection
import sklearn.preprocessing as preproc
from sklearn.feature_extraction import text
import pickle
import warnings
from sklearn.feature_extraction.text import CountVectorizer

In [27]:
training_data, test_data = sklearn.model_selection.train_test_split(data, train_size = 0.7, random_state=42)

In [28]:
print(training_data.shape)
print(test_data.shape)

(24056, 1)
(10311, 1)


In [31]:
bow_transform = CountVectorizer(tokenizer=lambda doc: doc, ngram_range=[3,3], lowercase=False) 